In [ ]:
import pandas as pd
import numpy as np
from copy import deepcopy

In [ ]:
class MobilePhone:
    #konstruktor
    def __init__(self, model, price, antutu_score, dxomark, ppi, battery_capacity):
        self.model = model
        self.price = price
        self.antutu_score = antutu_score
        self.dxomark = dxomark
        self.ppi = ppi
        self.battery_capacity = battery_capacity
    
    #printowanie telefonu
    def __str__(self):
        return f"Model: {self.model}, Price ($): {self.price}, Antutu Score: {self.antutu_score}, DXOMARK: {self.dxomark}, PPI: {self.ppi}, Battery Capacity (mAh): {self.battery_capacity}"

In [ ]:
#wczytaj smartfony i utwórz liste z objektami klasy MobilePhone
data = pd.read_csv("smartfony.csv")
phones = []
for index, row in data.iterrows():
    # phone = MobilePhone(row['Model'], row['Cena ($)'], row['Antutu Score'], row['DXOMARK'], row['ppi'], row['pojemność baterii (mAh)'])
    phones.append([row['Model'], int(row['Cena ($)']), int(row['Antutu Score']), int(row['DXOMARK']), int(row['ppi']), int(row['pojemność baterii (mAh)'])])
    # phones.append(phone)


In [ ]:
#wyswietl telefony
for phone in phones:
    print(phone)

In [ ]:
class Criterion:
    def __init__(self, name, type, weight, indiff, pref, veto):
        self.name = name
        self.type = type
        self.weight = weight
        self.indiff = indiff
        self.pref = pref
        self.veto = veto

# kryteria:
# cena w=2 q=50 p=200 v=600
# antutu w=4 q=100000 p=300000 v=500000
# dxomark w=4 q=10 p=20 v=40
# dpi w=2 q=20 p=50 v=100
# poj. bat. w=3 q=400 p=1200 v=2000


# Inicjalizacja kryteriów
criteria = []

criteria.append(["price", "cost", 2, 50, 200, 600])
criteria.append(["antutu", "gain", 4, 100000, 300000, 500000])
criteria.append(["dxomark", "gain", 4, 10, 20, 40])
criteria.append(["dpi", "gain", 2, 20, 50, 100])
criteria.append(["battery", "gain", 3, 400, 1200, 2000])

# criteria.append(Criterion("price", "cost", 2, 50, 200, 600))
# criteria.append(Criterion("antutu", "gain", 4, 100000, 300000, 500000))
# criteria.append(Criterion("dxomark", "gain", 4, 10, 20, 40))
# criteria.append(Criterion("dpi", "gain", 2, 20, 50, 100))
# criteria.append(Criterion("battery", "gain", 3, 400, 1200, 2000))



In [ ]:
# Obliczanie macierzy wiarygodności

concordance = np.ones(shape=(len(phones), len(phones)))

for i, phone1 in enumerate(phones):
    for j, phone2 in enumerate(phones):
        cis = [] 
        dis = []
        for k, criterion in enumerate(criteria):
            if(criterion[1] == "gain"):
                if phone1[k+1] - phone2[k+1] >= -criterion[3]:
                    cis.append(1)
                elif phone1[k+1] - phone2[k+1] < -criterion[4]:
                    cis.append(0)
                else:
                    cis.append((criterion[4] - (phone2[k+1] - phone1[k+1])) / (criterion[4] - criterion[3]))
                
                if phone1[k+1] - phone2[k+1] <= -criterion[5]:
                    dis.append(1)
                elif phone1[k+1] - phone2[k+1] >= -criterion[4]:
                    dis.append(0)
                else:
                    dis.append(((phone2[k+1] - phone1[k+1]) - criterion[4]) / (criterion[5] - criterion[4]))

            else:
                if phone1[k+1] - phone2[k+1] <= criterion[3]:
                    cis.append(1)
                elif phone1[k+1] - phone2[k+1] > criterion[4]:
                    cis.append(0)
                else:
                    cis.append((criterion[4] - (phone1[k+1] - phone2[k+1])) / (criterion[4] - criterion[3]))

                if phone1[k+1] - phone2[k+1] >= criterion[5]:
                    dis.append(1)
                elif phone1[k+1] - phone2[k+1] <= criterion[4]:
                    dis.append(0)
                else:
                    dis.append((criterion[5] - (phone1[k+1] - phone2[k+1])) / (criterion[5] - criterion[4]))

        c = 0
        weight_sum = 0

        for ci, criterion in zip(cis, criteria):
            c += ci * criterion[2]
            weight_sum += criterion[2]
        
        c /= weight_sum

        sigma = c

        for d in dis:
            if d > c:
                if c == 1:
                    print("c = 1")
                sigma = sigma * (1 - d) / (1 - c)
        
        concordance[i][j] = round(sigma, 2)


print(concordance)                 

In [ ]:
# def descending_destilation(matrix, variants, upper_treshold=None, depth=0):
#     ans = []
#     depth += 1


#     while True:
#         n = matrix.shape[0]

#         matrix_copy = matrix.copy()
#         if upper_treshold is None:
#             mask = np.eye(matrix.shape[0], dtype=bool)
#             masked_arr = np.ma.masked_array(matrix, mask)
#             upper_treshold = np.max(masked_arr)

#         if upper_treshold == 0:
#             if len(ans) == 1:
#                 return ans
#             else:
#                 return variants
            
#         if len(variants) == 0:
#             return ans
#         if len(variants) == 1:
#             return ans.append(variants)
        
#         s = -0.15 * upper_treshold + 0.3
#         lower_treshold = np.max(matrix[matrix < upper_treshold - s]) if matrix[matrix < upper_treshold - s].size > 0 else 0
#         for i in range(n):
#             for j in range(n):
#                 if matrix[i][j] > lower_treshold and matrix[i][j] > matrix[j][i] + s:
#                     continue
#                 else:
#                     matrix_copy[i][j] = 0
#         strength = np.array([np.count_nonzero(row) for row in matrix_copy])
#         weakness = np.array([np.count_nonzero(column) for column in matrix_copy.T])
#         quality = strength - weakness

#         print(quality)

#         if np.count_nonzero(quality == np.max(quality)) == 1:
#             max_index = np.argmax(quality)
#             ans.append(variants[max_index])
#             variants.pop(max_index)
#             matrix = np.delete(matrix, max_index, axis=0)
#             matrix = np.delete(matrix, max_index, axis=1)

#         else:
#             max_indices = np.where(quality == np.max(quality))[0].tolist()

#             internal_matrix = deepcopy(matrix)
#             internal_matrix = internal_matrix[max_indices][:, max_indices]
#             internal_variants = [variants[i] for i in max_indices]

#             internal_ans = descending_destilation(internal_matrix, internal_variants, lower_treshold, depth)

#             internal_indices = [variants.index(variant) for variant in internal_ans]
#             internal_indices = sorted(internal_indices, reverse=True)

#             for internal_index in internal_indices:
#                 variants.pop(internal_index)

#             matrix = np.delete(matrix, np.array(internal_indices), axis=0)
#             matrix = np.delete(matrix, np.array(internal_indices), axis=1)

#             ans.append(internal_ans)

#             if matrix.shape[0] == 0:
#                 return ans
#         print(ans)




# des_dest = descending_destilation(concordance, phones)